## DATASET CLEANING AND TRANSFORM

In [1]:
%pip install -U imbalanced-learn
%pip install scikit-learn==1.0.2


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None) #MOSTRAR TODAS LAS COLUMNAS
y_col_name = 'Protestas'

In [3]:
!gdown --id 1-tNBUCWFU0_b9QW6ie4Vvfs7eb4WbIPY
# https://drive.google.com/file/d/1-tNBUCWFU0_b9QW6ie4Vvfs7eb4WbIPY/view?usp=share_link
!gdown --id 11eGwBDC3FoJh8eanfpYXFAS-3SZ6vNnO
#https://drive.google.com/file/d/11eGwBDC3FoJh8eanfpYXFAS-3SZ6vNnO/view?usp=sharing

/usr/local/lib/python3.10/site-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (uriginal): https://drive.google.com/uc?id=1-tNBUCWFU0_b9QW6ie4Vvfs7eb4WbIPY
From (redirected): https://drive.google.com/uc?id=1-tNBUCWFU0_b9QW6ie4Vvfs7eb4WbIPY&confirm=t&uuid=f584c235-6b53-46e7-9be9-5d959caedcbc
To: /Users/matiaslopez/Documents/tesis/tesis-model/TestBalanceo1.csv
100%|████████████████████████████████████████| 827M/827M [01:57<00:00, 7.04MB/s]
/usr/local/lib/python3.10/site-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=11eGwBDC3FoJh8eanfpYXFAS-3SZ6vNnO
To: /Users/matiaslopez/Documents/tesis/tesis-model/OcidConBanderas.csv
100%|███████████████████████

In [4]:
df_completo = pd.read_csv('TestBalanceo1.csv', sep=";;;", index_col=False)
df_banderas_completo = pd.read_csv('OcidConBanderas.csv', sep=";", index_col=False)
df_banderas_completo.drop(['Unnamed: 10'], inplace=True, axis=1)
df_completo = df_completo.merge(df_banderas_completo, left_on='ocid', right_on='id', how='left')

/var/folders/0s/ngx4k9892vv59rf4q5h1h4w00000gn/T/ipykernel_1812/2578874532.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_completo = pd.read_csv('TestBalanceo1.csv', sep=";;;", index_col=False)


In [5]:
#Drop
df_dummizado = df_completo.drop(['ocid', 'id_x', 'id_y', 'tender.id', # Dropeamos por que son IDs
    'tender.bidOpening.address.streetAddress', 'tender.enquiriesAddress.streetAddress', #Dropeamos por que son direcciones, creemos que no tienen valor
    'ocid.1', 'tender.procurementIntention.uri', # URL y id
    'tender.procurementIntention.description', #Descripcion
    'tender.procurementIntention.category', # Dropeamos porque ya esta la info en otra columna
    'tender.procurementIntention.title', #Titulo, practicamente todos son unicos
    'initiationType', #Todos tienen el mismo valor (tender)
    'tender.procurementIntention.id', # Ver vien por que dropeamos
    'tag', #Todos tienen el mismo valor (compiled)
    'language', #Todos tienen el mismo valor (es)
    'tender.techniques.hasFrameworkAgreement', #Solo 48 rows con datos, no vale la pena
    # 'Protestas', #Tiene info del resultado
	'tender.contractPeriod.durationInDays',
	'has_complaint',
], axis=1)

#Dummies
df_dummizado = pd.get_dummies(df_dummizado, columns=['tender.status']) # Cardinalidad baja
df_dummizado = pd.get_dummies(df_dummizado, columns=['tender.awardCriteria']) # Cardinalidad baja
df_dummizado = pd.get_dummies(df_dummizado, columns=['tender.awardCriteriaDetails']) # Cardinalidad baja
df_dummizado = pd.get_dummies(df_dummizado, columns=['tender.statusDetails']) # Cardinalidad baja
df_dummizado = pd.get_dummies(df_dummizado, columns=['tender.hasEnquiries']) # Cardinalidad baja
df_dummizado = pd.get_dummies(df_dummizado, columns=['tender.value.currency']) # Cardinalidad baja
df_dummizado = pd.get_dummies(df_dummizado, columns=['tender.mainProcurementCategory']) # Cardinalidad baja
df_dummizado = pd.get_dummies(df_dummizado, columns=['tender.procurementMethod']) # Cardinalidad baja
df_dummizado = pd.get_dummies(df_dummizado, columns=['tender.procurementMethodRationale']) # Cardinalidad baja
df_dummizado = pd.get_dummies(df_dummizado, columns=['tender.procurementIntention.rationale']) # Cardinalidad baja
df_dummizado = pd.get_dummies(df_dummizado, columns=['planning.budget.amount.currency']) # Cardinalidad baja

df_dummizado = pd.get_dummies(df_dummizado, columns=['tender.procurementIntention.status']) # Cardinalidad baja
df_dummizado = pd.get_dummies(df_dummizado, columns=['tender.procurementIntention.statusDetails']) # Cardinalidad baja


#Dates
df_dummizado['tender.bidOpening.date.month'] = pd.to_datetime(df_dummizado['tender.bidOpening.date'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.month
df_dummizado['tender.bidOpening.date.year'] = pd.to_datetime(df_dummizado['tender.bidOpening.date'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.year
df_dummizado['tender.bidOpening.date.yearmonth'] = pd.to_datetime(df_dummizado['tender.bidOpening.date'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.strftime('%Y%m')

df_dummizado['tender.datePublished.month'] = pd.to_datetime(df_dummizado['tender.datePublished'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.month
df_dummizado['tender.datePublished.year'] = pd.to_datetime(df_dummizado['tender.datePublished'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.year
df_dummizado['tender.datePublished.yearmonth'] = pd.to_datetime(df_dummizado['tender.datePublished'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.strftime('%Y%m')

df_dummizado['tender.tenderPeriod.startDate.month'] = pd.to_datetime(df_dummizado['tender.tenderPeriod.startDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.month
df_dummizado['tender.tenderPeriod.startDate.year'] = pd.to_datetime(df_dummizado['tender.tenderPeriod.startDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.year
df_dummizado['tender.tenderPeriod.startDate.yearmonth'] = pd.to_datetime(df_dummizado['tender.tenderPeriod.startDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.strftime('%Y%m')

df_dummizado['tender.tenderPeriod.endDate.month'] = pd.to_datetime(df_dummizado['tender.tenderPeriod.startDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.month
df_dummizado['tender.tenderPeriod.endDate.year'] = pd.to_datetime(df_dummizado['tender.tenderPeriod.startDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.year
df_dummizado['tender.tenderPeriod.endDate.yearmonth'] = pd.to_datetime(df_dummizado['tender.tenderPeriod.startDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.strftime('%Y%m')

df_dummizado['tender.awardPeriod.startDate.month'] = pd.to_datetime(df_dummizado['tender.awardPeriod.startDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.month
df_dummizado['tender.awardPeriod.startDate.year'] = pd.to_datetime(df_dummizado['tender.awardPeriod.startDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.year
df_dummizado['tender.awardPeriod.startDate.yearmonth'] = pd.to_datetime(df_dummizado['tender.awardPeriod.startDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.strftime('%Y%m')

df_dummizado['tender.enquiryPeriod.endDate.month'] = pd.to_datetime(df_dummizado['tender.enquiryPeriod.endDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.month
df_dummizado['tender.enquiryPeriod.endDate.year'] = pd.to_datetime(df_dummizado['tender.enquiryPeriod.endDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.year
df_dummizado['tender.enquiryPeriod.endDate.yearmonth'] = pd.to_datetime(df_dummizado['tender.enquiryPeriod.endDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.strftime('%Y%m')

df_dummizado['tender.enquiryPeriod.startDate.month'] = pd.to_datetime(df_dummizado['tender.enquiryPeriod.startDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.month
df_dummizado['tender.enquiryPeriod.startDate.year'] = pd.to_datetime(df_dummizado['tender.enquiryPeriod.startDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.year
df_dummizado['tender.enquiryPeriod.startDate.yearmonth'] = pd.to_datetime(df_dummizado['tender.enquiryPeriod.startDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.strftime('%Y%m')

df_dummizado['date.month'] = pd.to_datetime(df_dummizado['date'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.month
df_dummizado['date.year'] = pd.to_datetime(df_dummizado['date'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.year
df_dummizado['date.yearmonth'] = pd.to_datetime(df_dummizado['date'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.strftime('%Y%m')

df_dummizado['planning.estimatedDate.month'] = pd.to_datetime(df_dummizado['planning.estimatedDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.month
df_dummizado['planning.estimatedDate.year'] = pd.to_datetime(df_dummizado['planning.estimatedDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.year
df_dummizado['planning.estimatedDate.yearmonth'] = pd.to_datetime(df_dummizado['planning.estimatedDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.strftime('%Y%m')
# Dropeando data interesante (Despues ver como procesar, son datos practicamente unicos)
df_dummizado = df_dummizado.drop(['tender.title', 'tender.submissionMethodDetails', 'tender.eligibilityCriteria', 'tender.mainProcurementCategoryDetails', 'planning.budget.description', ], axis=1)

# Dropeando por cardinalidad alta? Datos no 100% unicos por cada row
df_dummizado = df_dummizado.drop(['tender.procurementMethodDetails', 'tender.procuringEntity.id', 'tender.procuringEntity.name', 'buyer.id', 'buyer.name', 
	'tender.procurementIntention.procuringEntity.id', 'tender.procurementIntention.procuringEntity.name', 'secondStage.id',
], axis=1)

#Dropeando dates
df_dummizado = df_dummizado.drop(['tender.bidOpening.date',
	'tender.datePublished', 'tender.tenderPeriod.startDate', 'tender.tenderPeriod.endDate', 'tender.awardPeriod.startDate','tender.enquiryPeriod.endDate','tender.enquiryPeriod.startDate',
    'date', 'planning.estimatedDate', 
    'tender.contractPeriod.maxExtentDate', # Hay solo 7000
    'tender.procurementIntention.startDate', 'tender.procurementIntention.publishedDate', #Hay solo 1200 en todo el dataset
	'tender.contractPeriod.startDate', 'tender.contractPeriod.endDate' # Hay solo 35 datos en todo el dataset
], axis= 1)

# Otras conversiones
df_dummizado['tender.techniques.hasElectronicAuction'] = np.where(df_dummizado['tender.techniques.hasElectronicAuction'] == True, 1, 0)


In [6]:
# df_dummizado['tender.value.amount'] = df_dummizado['tender.value.amount'].fillna() #
# df_dummizado['tender.tenderPeriod.durationInDays'] = df_dummizado['tender.tenderPeriod.durationInDays'].fillna(-1)
# df_dummizado['tender.enquiryPeriod.durationInDays'] = df_dummizado['tender.enquiryPeriod.durationInDays'].fillna(-1)
# df_dummizado['tender.numberOfTenderers'] = df_dummizado['tender.numberOfTenderers'].fillna(-1)
# df_dummizado['planning.budget.amount.amount'] = df_dummizado['planning.budget.amount.amount'].fillna(-1)
# df_dummizado['tender.contractPeriod.durationInDays'] = df_dummizado['tender.contractPeriod.durationInDays'].fillna(-1)
# df_dummizado['tender.techniques.hasElectronicAuction'] = df_dummizado['tender.techniques.hasElectronicAuction'].fillna(False)

df_dummizado['tender.bidOpening.date.month'].fillna(0, inplace=True)
df_dummizado['tender.bidOpening.date.year'].fillna(0, inplace=True)
df_dummizado['tender.bidOpening.date.yearmonth'].fillna(0, inplace=True)

df_dummizado['tender.enquiryPeriod.endDate.month'].fillna(0, inplace=True)
df_dummizado['tender.enquiryPeriod.endDate.year'].fillna(0, inplace=True)
df_dummizado['tender.enquiryPeriod.endDate.yearmonth'].fillna(0, inplace=True)
df_dummizado['tender.enquiryPeriod.startDate.month'].fillna(0, inplace=True)
df_dummizado['tender.enquiryPeriod.startDate.year'].fillna(0, inplace=True)
df_dummizado['tender.enquiryPeriod.startDate.yearmonth'].fillna(0, inplace=True)
df_dummizado['tender.tenderPeriod.startDate.month'].fillna(0, inplace=True)
df_dummizado['tender.tenderPeriod.startDate.year'].fillna(0, inplace=True)
df_dummizado['tender.tenderPeriod.startDate.yearmonth'].fillna(0, inplace=True)
df_dummizado['tender.tenderPeriod.endDate.month'].fillna(0, inplace=True)
df_dummizado['tender.tenderPeriod.endDate.year'].fillna(0, inplace=True)
df_dummizado['tender.tenderPeriod.endDate.yearmonth'].fillna(0, inplace=True)
df_dummizado['tender.awardPeriod.startDate.month'].fillna(0, inplace=True)
df_dummizado['tender.awardPeriod.startDate.year'].fillna(0, inplace=True)
df_dummizado['tender.awardPeriod.startDate.yearmonth'].fillna(0, inplace=True)
df_dummizado['tender.datePublished.month'].fillna(0, inplace=True)
df_dummizado['tender.datePublished.year'].fillna(0, inplace=True)
df_dummizado['tender.datePublished.yearmonth'].fillna(0, inplace=True)
df_dummizado['planning.estimatedDate.month'].fillna(0, inplace=True)
df_dummizado['planning.estimatedDate.year'].fillna(0, inplace=True)
df_dummizado['planning.estimatedDate.yearmonth'].fillna(0, inplace=True)
df_dummizado['planning.identifier'] = df_dummizado['planning.identifier'].fillna(0)

# Drop if col NaN = 
df_dummizado = df_dummizado[df_dummizado['tender.value.amount'].notna()]

df_dummizado['tender.value.amount'].fillna(df_dummizado['tender.value.amount'].mean(), inplace=True)

# Tender duration y enquiry period dependen también del procurement method, para cada procurement method por ley hay unos mínimos, sugiero usar eso los mínimos. Investiguen esto si. 
df_dummizado['tender.tenderPeriod.durationInDays'].fillna(df_dummizado['tender.tenderPeriod.durationInDays'].mean(), inplace=True)
df_dummizado['tender.enquiryPeriod.durationInDays'].fillna(df_dummizado['tender.enquiryPeriod.durationInDays'].mean(), inplace=True)

df_dummizado['tender.numberOfTenderers'].fillna(1, inplace=True)
df_dummizado['planning.budget.amount.amount'].fillna(df_dummizado['planning.budget.amount.amount'].mean(), inplace=True)
df_dummizado['tender.techniques.hasElectronicAuction'] = df_dummizado['tender.techniques.hasElectronicAuction'].fillna(False)


In [7]:
df_complaints = df_dummizado[(df_dummizado[y_col_name] == True)]
df_no_complaints = df_dummizado[(df_dummizado[y_col_name] == False)]
# PERCENTAGE OF COMPLAINTS / NO COMPLAINTS
(df_complaints.shape[0] * 1) / df_no_complaints.shape[0]

0.09124175526307864

In [8]:
df_dummizado_up_to_2023 = df_dummizado[df_dummizado['date.year'] < 2023]
# df_dummizado = df_dummizado[df_dummizado['tender.value.amount'].notna()]

In [9]:
df_dummizado_from_2023 = df_dummizado[df_dummizado['date.year'] >= 2023]
df_dummizado_from_2023.shape

(11687, 790)

In [10]:
df_dummizado.shape

(144930, 790)

In [11]:
df_dummizado_up_to_2023.shape

(133243, 790)

In [12]:
df_complaints = df_dummizado_up_to_2023[(df_dummizado_up_to_2023[y_col_name] == True)]
df_no_complaints = df_dummizado_up_to_2023[(df_dummizado_up_to_2023[y_col_name] == False)]
# PERCENTAGE OF COMPLAINTS / NO COMPLAINTS
(df_complaints.shape[0] * 1) / df_no_complaints.shape[0]

0.07414185060380826

In [13]:
df_dummizado_2016_2022 = df_dummizado_up_to_2023[df_dummizado['date.year'] >= 2016]

/var/folders/0s/ngx4k9892vv59rf4q5h1h4w00000gn/T/ipykernel_1812/994191932.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_dummizado_2016_2022 = df_dummizado_up_to_2023[df_dummizado['date.year'] >= 2016]


In [14]:
df_up_to_2015 = df_dummizado_up_to_2023[df_dummizado['date.year'] < 2016]

/var/folders/0s/ngx4k9892vv59rf4q5h1h4w00000gn/T/ipykernel_1812/4189494223.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_up_to_2015 = df_dummizado_up_to_2023[df_dummizado['date.year'] < 2016]


In [15]:
df_cmp_up_to_2015 = df_up_to_2015[df_up_to_2015[y_col_name] == True]

In [16]:
# balanced_df = pd.concat([balanced_df_true, sample_false])
df_final_to_balance = pd.concat([df_cmp_up_to_2015, df_dummizado_2016_2022])

In [17]:
df_complaints = df_final_to_balance[(df_final_to_balance[y_col_name] == True)]
df_no_complaints = df_final_to_balance[(df_final_to_balance[y_col_name] == False)]
# PERCENTAGE OF COMPLAINTS / NO COMPLAINTS
(df_complaints.shape[0] * 1) / df_no_complaints.shape[0]

0.10936570980093704

## SMOTE BALANCING

In [25]:
from imblearn.over_sampling import SMOTE
from collections import Counter
X = df_final_to_balance.drop(y_col_name, axis=1)  # Features (all columns except 'target')
y = df_final_to_balance[y_col_name]  # Target variable
print('Original dataset shape %s' % Counter(y))
#https://imbalanced-learn.org/stable/references/generated/imblearn.over_sampling.SMOTE.html#examples-using-imblearn-over-sampling-smote
smote = SMOTE(random_state=24, sampling_strategy= 0.35)
X_resampled, y_resampled = smote.fit_resample(X, y)
print('Resampled dataset shape %s' % Counter(y_resampled))

Original dataset shape Counter({False: 84094, True: 9197})


/usr/local/lib/python3.10/site-packages/sklearn/utils/extmath.py:153: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


Resampled dataset shape Counter({False: 84094, True: 29432})


In [26]:
df_final_to_balance.to_csv('df_final_unbalanced.csv', index=False, encoding='utf-8')

In [27]:
df_dummizado_from_2023.to_csv('df_dummizado_from_2023.csv', index=False, encoding='utf-8')

In [28]:
resampled_df = pd.concat([X_resampled, y_resampled], axis=1, join="inner")
resampled_df.to_csv('balanced_output_35.csv', index = False, encoding='utf-8') # False: not include index

## USEFUL FUNCTIONS

In [29]:
# TO CSV EXAMPLE

X_resampled.to_csv('ouput_x.csv', index = False, encoding='utf-8') # False: not include index
y_resampled.to_csv('ouput_y.csv', index = False, encoding='utf-8') # False: not include index
# print(df)

In [30]:
# GROUP BY EXAMPLE
gkk = df_dummizado[df_dummizado['Protestas'] == True].groupby(['date.year'], dropna=False)
# df = df_dummizado[df_dummizado]
# gkk = df_dummizado.groupby(['date.year'], dropna=False)
gkk.count()

,tender.value.amount,tender.tenderPeriod.durationInDays,tender.enquiryPeriod.durationInDays,tender.numberOfTenderers,planning.identifier,planning.budget.amount.amount,tender.techniques.hasElectronicAuction,tender.lots.count,tender.enquiries.count,awards.count,contracts.count,contracts.guarantees.obligations_1,contracts.guarantees.obligations_2,contracts.guarantees.obligations_3,contracts.guarantees.obligations_4,contracts.guarantees.obligations_5,contracts.guarantees.obligations_6,contracts.guarantees.obligations_7,contracts.guarantees.obligations_8,contracts.guarantees.obligations_9,contracts.guarantees.obligations_10,contracts.guarantees.obligations_11,contracts.guarantees.obligations_12,contracts.guarantees.obligations_13,contracts.guarantees.obligations_14,contracts.guarantees.obligations_15,contracts.guarantees.obligations_16,contracts.guarantees.obligations_17,contracts.guarantees.obligations_18,contracts.investmentProjects.id q1,contracts.investmentProjects.id q2,contracts.investmentProjects.id q3,contracts.investmentProjects.id q4,contracts.amendments.amendsAmount_pyg,contracts.amendments.amendsAmount_usd,contracts.amendments.count,contracts.implementation.purchaseOrders.count,contracts.implementation.transactions.count,contracts.documents.DocumentTypeDetails_1,contracts.documents.DocumentTypeDetails_2,contracts.documents.DocumentTypeDetails_3,contracts.documents.DocumentTypeDetails_4,contracts.documents.DocumentTypeDetails_5,contracts.documents.DocumentTypeDetails_6,contracts.documents.DocumentTypeDetails_7,contracts.documents.DocumentTypeDetails_8,contracts.documents.DocumentTypeDetails_9,contracts.documents.DocumentTypeDetails_10,contracts.documents.DocumentTypeDetails_11,contracts.documents.DocumentTypeDetails_12,contracts.value.amount_pyg,contracts.value.amount_usd,contracts.status_1,contracts.status_2,contracts.status_3,contracts.status_4,contracts.statusDetails_1,contracts.statusDetails_2,contracts.statusDetails_3,contracts.statusDetails_4,contracts.statusDetails_5,contracts.statusDetails_6,contracts.statusDetails_7,contracts.statusDetails_8,contracts.statusDetails_9,awards.documents.DocumentTypeDetails_1,awards.documents.DocumentTypeDetails_2,awards.documents.DocumentTypeDetails_3,awards.documents.DocumentTypeDetails_4,awards.documents.DocumentTypeDetails_5,awards.documents.DocumentTypeDetails_6,awards.documents.DocumentTypeDetails_7,awards.documents.DocumentTypeDetails_8,awards.documents.DocumentTypeDetails_9,awards.documents.DocumentTypeDetails_10,awards.documents.DocumentTypeDetails_11,awards.documents.DocumentTypeDetails_12,awards.documents.DocumentTypeDetails_13,awards.documents.DocumentTypeDetails_14,awards.documents.DocumentTypeDetails_15,awards.documents.DocumentTypeDetails_16,awards.documents.DocumentTypeDetails_17,awards.documents.DocumentTypeDetails_18,awards.documents.DocumentTypeDetails_19,awards.documents.DocumentTypeDetails_20,awards.documents.DocumentTypeDetails_21,awards.suppliers.id q1,awards.suppliers.id q2,awards.suppliers.id q3,awards.suppliers.id q4,awards.value.amount_pyg,awards.value.amount_usd,awards.status_1,awards.status_2,awards.status_3,awards.status_4,awards.statusDetails_1,awards.statusDetails_2,awards.statusDetails_3,awards.statusDetails_4,awards.statusDetails_5,awards.statusDetails_6,awards.statusDetails_7,awards.statusDetails_8,awards.statusDetails_9,awards.statusDetails_10,awards.statusDetails_11,tender.coveredBy_1,tender.coveredBy_2,tender.coveredBy_3,tender.coveredBy_4,tender.coveredBy_5,tender.coveredBy_6,tender.coveredBy_7,tender.coveredBy_8,tender.notifiedSuppliers.id q1,tender.notifiedSuppliers.id q2,tender.notifiedSuppliers.id q3,tender.notifiedSuppliers.id q4,tender.tenderers.id q1,tender.tenderers.id q2,tender.tenderers.id q3,tender.tenderers.id q4,tender.items.classification.id.n5 q1,tender.items.classification.id.n5 q2,tender.items.classification.id.n5 q3,tender.items.classification.id.n5 q4,tender.items.classification.id.n4 q1,tender.items.classification.id.n4 q2,t

In [31]:
#VERIFICAR VALORES NAN EN DUMMIZADO

# list(df_dummizado.columns)
# df_dummizado.isna().mean().round(4) * 100
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
df_dummizado.isna().sum()

tender.value.amount                                          0
tender.tenderPeriod.durationInDays                           0
tender.enquiryPeriod.durationInDays                          0
tender.numberOfTenderers                                     0
planning.identifier                                          0
planning.budget.amount.amount                                0
tender.techniques.hasElectronicAuction                       0
tender.lots.count                                            0
tender.enquiries.count                                       0
awards.count                                                 0
contracts.count                                              0
contracts.guarantees.obligations_1                           0
contracts.guarantees.obligations_2                           0
contracts.guarantees.obligations_3                           0
contracts.guarantees.obligations_4                           0
contracts.guarantees.obligations_5                     